# Weather Dataset Preprocessing and feature Engineering

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val rawWeatherPath = s"${configuration.common.output.basePath}/common/data/raw_weather.parquet"
val weatherDF = spark.read.parquet(rawWeatherPath)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@11369e1b
session = org.apache.spark.sql.SparkSession@11369e1b
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

In [3]:
weatherDF.count()

16446741

In [4]:
weatherDF.printSchema

root
 |-- WBAN: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- StationType: integer (nullable = true)
 |-- SkyCondition: string (nullable = true)
 |-- SkyConditionFlag: string (nullable = true)
 |-- Visibility: double (nullable = true)
 |-- VisibilityFlag: string (nullable = true)
 |-- WeatherType: string (nullable = true)
 |-- WeatherTypeFlag: string (nullable = true)
 |-- DryBulbFarenheit: double (nullable = true)
 |-- DryBulbFarenheitFlag: string (nullable = true)
 |-- DryBulbCelsius: double (nullable = true)
 |-- DryBulbCelsiusFlag: string (nullable = true)
 |-- WetBulbFarenheit: double (nullable = true)
 |-- WetBulbFarenheitFlag: string (nullable = true)
 |-- WetBulbCelsius: double (nullable = true)
 |-- WetBulbCelsiusFlag: string (nullable = true)
 |-- DewPointFarenheit: double (nullable = true)
 |-- DewPointFarenheitFlag: string (nullable = true)
 |-- DewPointCelsius: double (nullable = true)
 |-- DewPointCelsiusFlag: string

In [5]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import scala.util.Try

## 1.Wind Features

In [7]:
weatherDF.select(
 "WindSpeed",
 "WindSpeedFlag",
 "WindDirection",
 "WindDirectionFlag",
 "ValueForWindCharacter",
 "ValueForWindCharacterFlag"
).show(5)


+---------+-------------+-------------+-----------------+---------------------+-------------------------+
|WindSpeed|WindSpeedFlag|WindDirection|WindDirectionFlag|ValueForWindCharacter|ValueForWindCharacterFlag|
+---------+-------------+-------------+-----------------+---------------------+-------------------------+
|      7.0|             |        130.0|                 |                     |                         |
|      3.0|             |        130.0|                 |                     |                         |
|      7.0|             |        140.0|                 |                     |                         |
|      8.0|             |        130.0|                 |                     |                         |
|      7.0|             |        100.0|                 |                     |                         |
+---------+-------------+-------------+-----------------+---------------------+-------------------------+
only showing top 5 rows



In [15]:
// WindSpeed avec comptage
weatherDF
  .groupBy("WindSpeed")
  .count()
  .orderBy(desc("WindSpeed"))
  .show(100)

+---------+-------+
|WindSpeed|  count|
+---------+-------+
|    172.0|      1|
|    125.0|      1|
|    115.0|      1|
|    113.0|      1|
|    109.0|      1|
|    105.0|      2|
|    100.0|      1|
|     99.0|      1|
|     98.0|      1|
|     94.0|      1|
|     93.0|      2|
|     92.0|      1|
|     91.0|      3|
|     90.0|      1|
|     89.0|      2|
|     87.0|      1|
|     86.0|      3|
|     83.0|      3|
|     82.0|      2|
|     81.0|      1|
|     79.0|      1|
|     78.0|      3|
|     76.0|      5|
|     75.0|      1|
|     74.0|      1|
|     72.0|      2|
|     70.0|      1|
|     69.0|      2|
|     68.0|      6|
|     67.0|      6|
|     66.0|     11|
|     65.0|      1|
|     64.0|      9|
|     63.0|     15|
|     62.0|     13|
|     61.0|     13|
|     60.0|     16|
|     59.0|     17|
|     57.0|     23|
|     56.0|     21|
|     55.0|     31|
|     54.0|     34|
|     53.0|     53|
|     52.0|     49|
|     51.0|     81|
|     49.0|    109|
|     48.0|    114|


In [16]:
// WindDirection avec comptage
weatherDF
  .groupBy("WindDirection")
  .count()
  .orderBy(desc("WindDirection"))
  .show(400)

+-------------+-------+
|WindDirection|  count|
+-------------+-------+
|        360.0| 178620|
|        359.0|    317|
|        358.0|    324|
|        357.0|    352|
|        356.0|    372|
|        355.0|    377|
|        354.0|    345|
|        353.0|    327|
|        352.0|    329|
|        351.0|    331|
|        350.0| 177200|
|        349.0|    321|
|        348.0|    303|
|        347.0|    296|
|        346.0|    307|
|        345.0|    355|
|        344.0|    323|
|        343.0|    287|
|        342.0|    281|
|        341.0|    340|
|        340.0| 187176|
|        339.0|    293|
|        338.0|    279|
|        337.0|    291|
|        336.0|    300|
|        335.0|    354|
|        334.0|    289|
|        333.0|    302|
|        332.0|    266|
|        331.0|    268|
|        330.0| 198260|
|        329.0|    322|
|        328.0|    305|
|        327.0|    306|
|        326.0|    334|
|        325.0|    375|
|        324.0|    321|
|        323.0|    305|
|        322.0| 

In [11]:
// ValueForWindCharacter avec comptage
weatherDF
  .groupBy("ValueForWindCharacter")
  .count()
  .orderBy(desc("count"))
  .show(999)

+---------------------+--------+
|ValueForWindCharacter|   count|
+---------------------+--------+
|                     |15097108|
|                   18|  124442|
|                   17|  120081|
|                   20|  119105|
|                   21|  115190|
|                   22|  106741|
|                   16|  100427|
|                   23|   99683|
|                   24|   88495|
|                   25|   79143|
|                   26|   67838|
|                   28|   56705|
|                   29|   47869|
|                   30|   39663|
|                   31|   32531|
|                   32|   27200|
|                   33|   22686|
|                   34|   18666|
|                   36|   14717|
|                   37|   11842|
|                   38|    9600|
|                   39|    7610|
|                   40|    6417|
|                   41|    5208|
|                   43|    4124|
|                   44|    3398|
|                   45|    2769|
|         